In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hebaahmed19","key":"5e703347a79548dd58b21839a4c10214"}\n'}

In [0]:
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets list
!kaggle competitions download -c new-york-city-taxi-fare-prediction
!ls

In [0]:
!pip install -U -q PyDrive
from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')

    100% |████████████████████████████████| 993kB 21.3MB/s 
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


dataPreparation = drive.CreateFile({'id':'1JRdDHbTpzHD5A63bLbG8vYvPbMY3Kqi8'})
dataPreparation.GetContentFile('dataPreparation.py')

In [0]:
#@Run only once to upload DataSet
uploaded = drive.CreateFile({'title': 'train.csv.zip'})
uploaded.SetContentFile('train.csv.zip')
uploaded.Upload()


uploaded = drive.CreateFile({'title': 'test.csv'})
uploaded.SetContentFile('test.csv')
uploaded.Upload()

FileNotFoundError: ignored

In [0]:
!unzip -q /content/gdrive/My\ Drive/TaxiFare/data/train.csv.zip 

In [0]:
import numpy as np # linear algebra
import pandas as pd # CSV file I/O (e.g. pd.read_csv)
import dataPreparation


train,test = dataPreparation.readAndCleanData('./train.csv','/content/gdrive/My Drive/TaxiFare/data/test.csv')
print("Shape of Training Data after cleaning ",train.shape)
print("Shape of Testing Data after cleaning", test.shape)

# TODO: Call function for dataAnalysis here.

# For ( heba & fatema but sobhy & feryal will use train & test above only)
# For training and testing using models.
xTrain, xTest, yTrain, yTest \
=dataPreparation.prepareDataForModel(train,'fare_amount',dropCols=['key','pickup_datetime'],isTrain=True,split=0.2)


testData=dataPreparation.prepareDataForModel(test,'fare_amount',dropCols=['key','pickup_datetime'],isTrain=False)


/content/dataPreparation.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['pickup_datetime']=pd.to_datetime(data['pickup_datetime'],format='%Y-%m-%d %H:%M:%S UTC')
/content/dataPreparation.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['pickup_day']=data['pickup_datetime'].apply(lambda x:x.day)
/content/dataPreparation.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pand

Shape of Training Data after cleaning  (2935680, 13)
Shape of Testing Data after cleaning (9914, 12)
Shape of Training Features (2348544, 10)
Shape of Validation Features  (587136, 10)
Shape of Test Data (9914, 10)


In [0]:
from sklearn.metrics import mean_squared_error
avg_fare=round(np.mean(yTrain),2) #11.31
baseline_pred=np.repeat(avg_fare,yTest.shape[0])
baseline_rmse=np.sqrt(mean_squared_error(baseline_pred, yTest))
print("Basline RMSE of Validation data :",baseline_rmse)

baseline_pred=np.repeat(avg_fare,yTrain.shape[0])
baseline_train_rmse=np.sqrt(mean_squared_error(baseline_pred,yTrain))
print("Train RMSE for Basline is ", baseline_train_rmse)

variance=baseline_train_rmse - baseline_rmse
print("Variance of Basline is ", variance)

Basline RMSE of Validation data : 9.611176143919515
Train RMSE for Basline is  9.654032032143839
Variance of Basline is  0.042855888224323735


In [0]:
from sklearn.linear_model import LinearRegression

lm = LinearRegression()
lm.fit(xTrain,yTrain)
yPred=np.round(lm.predict(xTest),2)
lm_rmse=np.sqrt(mean_squared_error(yPred, yTest))
print("RMSE for Linear Regression is ",lm_rmse)

lm_train_rmse=np.sqrt(mean_squared_error(lm.predict(xTrain),yTrain))
print("Train RMSE for Linear Regression is ", lm_train_rmse)

variance=lm_train_rmse - lm_rmse
print("Variance of Linear Regression is ", variance)

RMSE for Linear Regression is  8.14028848424604
Train RMSE for Linear Regression is  8.209540778755612
Variance of Linear Regression is  0.06925229450957104


In [0]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 100, random_state = 883,n_jobs=-1, warm_start=True)


In [0]:
inc = int(xTrain.shape[0]/2)
ma = int(xTrain.shape[0])
for idx in range(0, ma,inc):
  x=xTrain.iloc[idx:min(xTrain.shape[0],idx+inc),:]
  y=yTrain.iloc[idx:min(yTrain.shape[0],idx+inc)]
  rf.fit(x,y)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:308: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:308: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "


In [0]:
rf.fit(xTrain,yTrain)

In [0]:
rf_pred= rf.predict(xTest)
rf_rmse=np.sqrt(mean_squared_error(rf_pred, yTest))
print("RMSE for Random Forest is ",rf_rmse)

rf_train_rmse=np.sqrt(mean_squared_error(rf.predict(xTrain),yTrain))
print("Train RMSE for Random Forest is ", rf_train_rmse)

variance=rf_train_rmse - rf_rmse
print("Variance of Random Forest is ", variance)

In [0]:
import time
print(int(xTrain.shape[0]*.92))
start = time.time()
x=xTrain.iloc[0:int(xTrain.shape[0]*.92),:]
y=yTrain.iloc[0:int(yTrain.shape[0]*.92)]
rf.fit(x,y)
end = time.time()
print(end - start)

NameError: ignored

In [0]:
start = time.time()
rf_pred= rf.predict(xTest)
end = time.time()
print(end - start)

rf_rmse=np.sqrt(mean_squared_error(rf_pred, yTest))
print("RMSE for Random Forest is ",rf_rmse)

rf_train_rmse=np.sqrt(mean_squared_error(rf.predict(x),y))
print("Train RMSE for Random Forest is ", rf_train_rmse)

variance=rf_train_rmse - rf_rmse
print("Variance of Random Forest is ", variance)

RMSE for Random Forest is  3.82622260380397
Train RMSE for Random Forest is  3.278367489839058
Variance of Random Forest is  -0.5478551139649119
